<a href="https://colab.research.google.com/github/mabench-tuc/LoRA-of-LLMs/blob/main/LoRA_adapted_Gpt_2_on_E2E_NLG_with_multiple_ranks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup Installation Process

In [ ]:
#!pip install git+https://github.com/microsoft/LoRA
!pip install -qU bitsandbytes accelerate loralib transformers peft trl
!pip install "datasets<4.0.0"
! pip install -U sacrebleu evaluate rouge-score
#!pip install -U datasets

In [ ]:
import torch
torch.cuda.is_available()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM, TrainingArguments
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoModelForSequenceClassification
from datasets import load_dataset
import bitsandbytes as bnb

In [ ]:
import time
import math
import json
import torch
import numpy as np
import pandas as pd
import evaluate

## Model's Loading
Here we load the model with its weights and the tokenizer

## Load the GPT-2 Large model

In [ ]:
# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the GPT-2 Large model and tokenizer
print("Loading gpt2-large model...")
gpt2_large_model = AutoModelForCausalLM.from_pretrained("gpt2-large").to(device)

gpt2_large_tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
print("Successfully loaded gpt2-large model.")

OUTPUT_DIR = "./lora_e2e_mranks_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
model=gpt2_large_model
tokenizer= gpt2_large_tokenizer

In [ ]:
print(model)
#print_trainable_parameters(gpt2_large_model)

## Post-processing on the model
### Freezing the original weights
We need to apply some post-processing on the n-bit model to enable training, let's freeze all our layers, and cast the layer-norm in floatm for stability.

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

###Display Trainable Parameters

In [ ]:
def print_trainable_parameters(model):

    #Prints the number of trainable parameters in the model.

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

## Load E2E NLG Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("GEM/e2e_nlg")

# Add padding token for GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Tokenize (dynamic padding instead of fixed 512)
tokenized_datasets = dataset.map(
    lambda x: tokenizer(x["meaning_representation"], truncation=True, padding="longest"),
    batched=True
)
# Display an example of the tokenized dataset
print(tokenized_datasets["train"][0])

In [ ]:
tokenized_datasets.keys()

###Tokenization of the dataset

In [ ]:
# GPT-2-specific settings: Add padding tokens, as GPT-2 does not use padding by default
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 uses <|endoftext|> as a padding token

# Step 3: Define the tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["meaning_representation"],           # The "data" column contains the text in the E2E NLG dataset
        max_length=512,             # Max sequence length for GPT-2
        truncation=True,            # Truncate sequences longer than 512 tokens
        padding="max_length"        # Pad sequences shorter than 512 tokens
    )

# Step 4: Tokenize the dataset
tokenized_e2e_dataset = dataset.map(tokenize_function, batched=True)

# Step 5: Display an example of the tokenized dataset
print(tokenized_e2e_dataset["train"][0])

In [ ]:
#tokenized_dataset= tokenized_e2e_dataset
tokenized_dataset= tokenized_datasets
#tokenized_e2e_dataset.keys()

In [ ]:
# See available splits
print(dataset)

# Print the column headers for the training split
print("Column headers:", dataset["train"].column_names)

We create a smaller subset of the full dataset to fine-tune our model

In [ ]:
train_data= tokenized_dataset["train"].shuffle(seed=42).select(range(10000))
val_data= tokenized_dataset["validation"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

##Training Process

In [ ]:
#Import the necessary modules from the transformers library
import transformers
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
import evaluate
from rouge_score import rouge_scorer

###Train LoRA Adapter

In [ ]:
# filepath: experiments/lora_rank_sweep_eval.py

"""
Run a LoRA rank sweep, train each model, evaluate with BLEU + ROUGE, and save CSV:
Columns: rank, # of trainable parameters, bleu, rouge1, rouge2, rougeL
Assumptions:
 - `train_data`, `val_data`, `tokenized_dataset["validation"]`, and `tokenizer` exist in the environment.
 - required packages: torch, transformers, datasets, peft, accelerate (optional), nltk, rouge_score, pandas
"""

from typing import Tuple, Dict, List
import torch
import pandas as pd
from tqdm import tqdm

from transformers import AutoModelForCausalLM, TrainingArguments
# If using SFTTrainer from trl or a custom package, adapt import accordingly:
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# ---- USER VARIABLES / PREPARATION ----
# Ensure these variables are present in your environment:
# tokenizer, train_data, val_data, tokenized_dataset
# If not present, load/create them before running this script.

# This script expects the datasets to be huggingface datasets or lists of dicts with keys:
# For evaluation dataset: each item must have "input_text" and "target_text"


def preprocess_data(example):
    """Maps existing dataset columns to 'input_text' and 'target_text'."""
    # Assuming the tokenized dataset has 'meaning_representation' and 'target' columns
    return {
        "input_text": example["meaning_representation"],
        "target_text": example["target"], # Assuming 'target' is the correct column name for human reference
    }

# Create processed_data from the tokenized dataset splits
processed_data = {}
# Assuming 'tokenized_dataset' is a DatasetDict with 'validation' and 'test' splits
processed_data["validation"] = tokenized_dataset["validation"].map(preprocess_data)
processed_data["test"] = tokenized_dataset["test"].map(preprocess_data)


# ---- EVALUATION FUNCTION ----
def evaluate_model(model, tokenizer, dataset, device: str = None, max_gen_len: int = 100) -> Tuple[float, Dict[str, float]]:
    """
    Given a causal LM (model) and tokenizer, iterate dataset of dicts with keys:
    'input_text' and 'target_text', generate predictions, compute average BLEU and ROUGE scores.
    Returns: (avg_bleu, avg_rouge_dict)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)
    model.eval()

    smoothing = SmoothingFunction().method1
    bleu_scores: List[float] = []
    rouge_scores_list: List[Dict[str, float]] = []

    rouge_scorer_instance = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    # If dataset is a Dataset object, iterate with .select or plain iteration works
    for example in tqdm(dataset, desc="Evaluating", leave=False):
        input_text = example["input_text"]
        target_text = example["target_text"]

        # Tokenize input for generation
        enc = tokenizer(
            input_text,
            truncation=True,
            padding=True,
            return_tensors="pt",
            max_length=512
        )
        input_ids = enc.input_ids.to(device)
        attention_mask = enc.attention_mask.to(device) if hasattr(enc, "attention_mask") else None

        with torch.no_grad():
            gen_kwargs = {"max_new_tokens": max_gen_len, "num_beams": 5, "early_stopping": True, "pad_token_id": tokenizer.eos_token_id}
            if attention_mask is not None:
                output_ids = model.generate(input_ids, attention_mask=attention_mask, **gen_kwargs)
            else:
                output_ids = model.generate(input_ids, **gen_kwargs)

        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
        ref = target_text.strip()

        # BLEU (sentence-level)
        try:
            bleu_score = sentence_bleu([ref.split()], prediction.split(), smoothing_function=smoothing)
        except Exception:
            bleu_score = 0.0
        bleu_scores.append(bleu_score)

        # ROUGE
        rouge = rouge_scorer_instance.score(ref, prediction)
        rouge_scores_list.append({
            "rouge1": rouge["rouge1"].fmeasure,
            "rouge2": rouge["rouge2"].fmeasure,
            "rougeL": rouge["rougeL"].fmeasure,
        })

    # Avoid division by zero
    n = max(1, len(bleu_scores))
    avg_bleu = sum(bleu_scores) / n
    avg_rouge = {
        "rouge1": sum(r["rouge1"] for r in rouge_scores_list) / max(1, len(rouge_scores_list)),
        "rouge2": sum(r["rouge2"] for r in rouge_scores_list) / max(1, len(rouge_scores_list)),
        "rougeL": sum(r["rougeL"] for r in rouge_scores_list) / max(1, len(rouge_scores_list)),
    }

    return avg_bleu, avg_rouge


# ---- SWEEP & TRAIN ----
def run_lora_rank_sweep(
    ranks,
    tokenizer,
    train_data,
    val_data,
    processed_validation_dataset,
    output_csv: str = "lora_rank_results.csv",
    reduce_val_size: int = 500
):
    results = []

    for r in ranks:
        print(f"\n===== Training with LoRA rank {r} =====\n")

        config = LoraConfig(
            r=r,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["attn.c_attn"],
            bias="none",
            task_type="CAUSAL_LM",
        )

        # Load fresh base model for each rank
        base_model = AutoModelForCausalLM.from_pretrained("gpt2")
        base_model.config.pad_token_id = base_model.config.eos_token_id

        lora_model = get_peft_model(base_model, config)

        # Training args — adjust as needed
        training_args = TrainingArguments(
            output_dir=f"./results/rank_{r}",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=2e-4,
            lr_scheduler_type="linear",
            warmup_steps=500,
            num_train_epochs=5,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,
            gradient_checkpointing=True,
            fp16=torch.cuda.is_available(),
            dataloader_pin_memory=True,
            weight_decay=0.01,
            logging_dir="./logs_lorarank_gpt2",
            logging_strategy="steps",
            logging_steps=20,
            save_total_limit=2,
            load_best_model_at_end=True,
            report_to="none",
            optim="adamw_torch",
        )

        # Prepare train/val tokenized datasets for SFTTrainer expected format:
        # create labels column for causal LM training (copy of input_ids)
        train_with_labels = train_data.add_column("labels", train_data["input_ids"])
        val_small = val_data.select(range(min(reduce_val_size, len(val_data))))
        val_with_labels = val_small.add_column("labels", val_small["input_ids"])

        # If SFTTrainer is available, use it. If not, adapt to trainer class.
        try:
            trainer = SFTTrainer(
                model=lora_model,
                args=training_args,
                train_dataset=train_with_labels,
                eval_dataset=val_with_labels,
                compute_metrics=None,  # we'll compute BLEU/ROUGE separately below
            )

            trainer.train()
            # trainer.evaluate() can be left or used; we will compute BLEU/ROUGE using our evaluator for consistency.
            _ = trainer.evaluate()
        except NameError:
            # If SFTTrainer is not available, skip training (or integrate your own trainer here)
            print("SFTTrainer not found in this environment. Skipping trainer.train().")
        except Exception as e:
            print(f"Trainer failed with exception: {e}. Attempting to continue to evaluation.")

        # Count trainable parameters
        trainable_count = int(sum(p.numel() for p in lora_model.parameters() if p.requires_grad))

        # Evaluate on processed_validation_dataset (we'll use a reduced subset for speed)
        # If processed_validation_dataset is huge, select a smaller slice:
        try:
            val_for_eval = processed_validation_dataset.select(range(min(500, len(processed_validation_dataset))))
        except Exception:
            # If .select isn't available, assume it's an iterable and slice
            val_for_eval = list(processed_validation_dataset)[:500]

        avg_bleu, avg_rouge = evaluate_model(lora_model, tokenizer, val_for_eval)

        results.append({
            "rank": r,
            "# of trainable parameters": trainable_count,
            "bleu": avg_bleu,
            "rouge1": avg_rouge["rouge1"],
            "rouge2": avg_rouge["rouge2"],
            "rougeL": avg_rouge["rougeL"],
        })

        # Cleanup
        del lora_model, base_model
        if 'trainer' in locals():
            try:
                del trainer
            except Exception:
                pass
        torch.cuda.empty_cache()

    # Save results to CSV
    df = pd.DataFrame(results, columns=["rank", "# of trainable parameters", "bleu", "rouge1", "rouge2", "rougeL"])
    df.to_csv(output_csv, index=False)
    print(f"Saved LoRA rank sweep results to {output_csv}")
    return df


# ---- MAIN EXECUTION ----
if __name__ == "__main__":
    ranks = [1, 4, 8, 16, 32]
    # Ensure the following variables are defined in your session:
    # tokenizer, train_data, val_data, processed_data["validation"]
    # If they are not, load them here.

    # Ensure processed_data is created before calling run_lora_rank_sweep
    # This block should be executed before the sweep function call
    def preprocess_data_main(example):
        """Maps existing dataset columns to 'input_text' and 'target_text' for the main block."""
        # Assuming the tokenized dataset has 'meaning_representation' and 'target' columns
        return {
            "input_text": example["meaning_representation"],
            "target_text": example["target"], # Assuming 'target' is the correct column name
        }

    # Create processed_data from the tokenized dataset splits
    # Assuming 'tokenized_dataset' is a DatasetDict with 'validation' and 'test' splits
    processed_data = {}
    processed_data["validation"] = tokenized_dataset["validation"].map(preprocess_data_main)
    processed_data["test"] = tokenized_dataset["test"].map(preprocess_data_main)


    df_results = run_lora_rank_sweep(
        ranks=ranks,
        tokenizer=tokenizer,
        train_data=train_data,
        val_data=val_data,
        processed_validation_dataset=processed_data["validation"], # Use the correctly processed dataset
        output_csv="lora_rank_results.csv",
        reduce_val_size=500,
    )

    print(df_results)

##GPU Check

In [ ]:
!nvidia-smi

## Visualizations of LoRA Adaptation of GPT-2 on E2E NLG

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load results file (adjust path if needed)
df = pd.read_csv("lora_rank_results.csv")

plt.figure(figsize=(8,6))

# Plot all metrics
plt.plot(df['rank'], df['bleu'], marker='o', color='C0', label='BLEU')
plt.plot(df['rank'], df['rouge1'], marker='s', color='C1', label='ROUGE-1')
plt.plot(df['rank'], df['rouge2'], marker='^', color='C2', label='ROUGE-2')
plt.plot(df['rank'], df['rougeL'], marker='v', color='C3', label='ROUGE-L')

# Annotate each point with six decimal values
for i, r in enumerate(df['rank']):
    plt.text(r, df['bleu'][i], f"{df['bleu'][i]:.6f}", ha='center', va='bottom', fontsize=8, color='C0')
    plt.text(r, df['rouge1'][i], f"{df['rouge1'][i]:.6f}", ha='center', va='bottom', fontsize=8, color='C1')
    plt.text(r, df['rouge2'][i], f"{df['rouge2'][i]:.3f}", ha='center', va='bottom', fontsize=8, color='C2')
    plt.text(r, df['rougeL'][i], f"{df['rougeL'][i]:.6f}", ha='center', va='bottom', fontsize=8, color='C3')

# Labels and formatting
plt.xlabel("LoRA Rank", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.title("BLEU and ROUGE Metrics vs LoRA Rank", fontsize=14)
plt.xticks(df['rank'])
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig("bleu_rouge_vs_lora_ranks.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


plt.figure(figsize=(8,6))

# Plot each metric
plt.plot(df['# of trainable parameters'], df['bleu'], marker='o', label='BLEU', color='C0')
plt.plot(df['# of trainable parameters'], df['rouge1'], marker='s', label='ROUGE-1', color='C1')
plt.plot(df['# of trainable parameters'], df['rouge2'], marker='^', label='ROUGE-2', color='C2')
plt.plot(df['# of trainable parameters'], df['rougeL'], marker='v', label='ROUGE-L', color='C3')

# Annotate each point with six decimal places
for i, row in df.iterrows():
    plt.text(row['# of trainable parameters'], row['bleu'], f"{row['bleu']:.6f}", fontsize=8, color='C0', ha='left', va='bottom')
    plt.text(row['# of trainable parameters'], row['rouge1'], f"{row['rouge1']:.6f}", fontsize=8, color='C1', ha='left', va='bottom')
    plt.text(row['# of trainable parameters'], row['rouge2'], f"{row['rouge2']:.3f}", fontsize=8, color='C2', ha='left', va='bottom')
    plt.text(row['# of trainable parameters'], row['rougeL'], f"{row['rougeL']:.6f}", fontsize=8, color='C3', ha='left', va='bottom')

# Formatting
plt.xlabel("Number of Trainable Parameters", fontsize=12)
plt.ylabel("Score", fontsize=12)
plt.title("BLEU and ROUGE Metrics vs Trainable Parameters", fontsize=14)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

# Save figure
plt.savefig("combined_bleu_rouge_vs_params.png", dpi=300)
plt.show()


In [ ]:
# Metric efficiency vs LoRA rank: BLEU/ROUGE per million params + composite efficiency
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams.update({"font.size": 12})

fn = "lora_rank_results.csv"
df = pd.read_csv(fn)

# --- required columns check ---
req = ["rank", "# of trainable parameters", "bleu", "rouge1", "rouge2", "rougeL"]
missing = [c for c in req if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in {fn}: {missing}. Rename them or update the CSV.")

# --- prepare quantities ---
df = df.copy()
df["params_million"] = df["# of trainable parameters"].astype(float) / 1e6
# avoid division by zero
if (df["params_million"] <= 0).any():
    raise ValueError("Found non-positive values in '# of trainable parameters'")

# efficiencies: score per million parameters
df["eff_bleu"]   = df["bleu"].astype(float)   / df["params_million"]
df["eff_rouge1"] = df["rouge1"].astype(float) / df["params_million"]
df["eff_rouge2"] = df["rouge2"].astype(float) / df["params_million"]
df["eff_rougeL"] = df["rougeL"].astype(float) / df["params_million"]

# composite efficiency: normalize each metric by its max across ranks (range 0..1), sum, then divide by params_million
eps = 1e-12
norm_bleu   = df["bleu"].astype(float)   / (df["bleu"].astype(float).max()   + eps)
norm_r1     = df["rouge1"].astype(float) / (df["rouge1"].astype(float).max() + eps)
norm_r2     = df["rouge2"].astype(float) / (df["rouge2"].astype(float).max() + eps)
norm_rL     = df["rougeL"].astype(float) / (df["rougeL"].astype(float).max() + eps)
df["composite_norm"] = norm_bleu + norm_r1 + norm_r2 + norm_rL   # range ~0..4
df["eff_composite"] = df["composite_norm"] / df["params_million"]

# sort by rank for plotting
df.sort_values(by="rank", inplace=True)
ranks = df["rank"].values

# --- plotting ---
fig, ax = plt.subplots(figsize=(9,6))

# plot individual efficiencies
ax.plot(ranks, df["eff_bleu"],   marker='o', linestyle='-', label='BLEU per M params',   zorder=4)
ax.plot(ranks, df["eff_rouge1"], marker='s', linestyle='-', label='ROUGE-1 per M params', zorder=4)
ax.plot(ranks, df["eff_rouge2"], marker='^', linestyle='-', label='ROUGE-2 per M params', zorder=4)
ax.plot(ranks, df["eff_rougeL"], marker='v', linestyle='-', label='ROUGE-L per M params', zorder=4)

# plot composite efficiency on secondary axis (keeps scale readable)
ax2 = ax.twinx()
ax2.plot(ranks, df["eff_composite"], marker='D', linestyle='--', color='black', label='Composite efficiency', zorder=5)

# annotate each point with six decimal places (main metrics)
for i, r in enumerate(ranks):
    x = r
    # annotate the composite on the right axis with 6 decimals
    ax2.text(x, df["eff_composite"].iloc[i], f"{df['eff_composite'].iloc[i]:.6f}",
             fontsize=9, ha='left', va='bottom', color='black')
    # small annotations for BLEU (example) — avoid overcrowding by offsetting slightly
    ax.text(x, df["eff_bleu"].iloc[i], f"{df['eff_bleu'].iloc[i]:.6f}",
            fontsize=8, ha='right', va='bottom', color=ax.get_lines()[0].get_color())

# axis labels, title, ticks
ax.set_xlabel("LoRA rank", fontsize=13)
ax.set_ylabel("Score per million parameters (BLEU / ROUGE)", fontsize=13)
ax2.set_ylabel("Composite normalized score per million parameters", fontsize=13)
ax.set_title("Parameter-efficiency of LoRA ranks: per-metric and composite measures", fontsize=14)

ax.set_xticks(ranks)
ax.grid(True, linestyle='--', alpha=0.4)

# legends: combine handles from both axes
h1, l1 = ax.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax.legend(h1 + h2, l1 + l2, loc='upper right', fontsize=10)

plt.tight_layout()
plt.savefig("efficiency_vs_rank.png", dpi=300)
plt.show()

# --- brief printed summary for thesis reporting ---
print("Rank | params (M) | eff_bleu | eff_rougeL | eff_composite")
for _, row in df.iterrows():
    print(f"{int(row['rank']):>4} | {row['params_million']:.3f} M | "
          f"{row['eff_bleu']:.6f} | {row['eff_rougeL']:.6f} | {row['eff_composite']:.6f}")


##Inference for NLG Tasks

In [ ]:
# Load the tokenizer and model
tokenizergpt2 = GPT2Tokenizer.from_pretrained("gpt2")
# Load the tokenizer and model

modelgpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
tokenizer=tokenizergpt2
model= modelgpt2

In [ ]:
# Prompt
prompt = "Once upon a time"
#prompt= "Write a Python function to check if a number is prime."

# Tokenize the input text
inputs = tokenizer(prompt, return_tensors="pt")
inputs

In [ ]:
# Generate text
output_ids = model.generate(
    inputs.input_ids.to(model.device),
    attention_mask=inputs.attention_mask.to(model.device),
    pad_token_id=tokenizer.eos_token_id,
    max_length=50,
    num_return_sequences=1
)

output_ids

In [ ]:
# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)